In [46]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from pathlib import Path
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import lightgbm as lgb

# Cribbed from
# https://www.analyticsvidhya.com/blog/2021/08/complete-guide-on-how-to-use-lightgbm-in-python/

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# Load
df = pd.read_csv("dataset.csv", index_col='id')
df.assigned_schedule = df.assigned_schedule.astype('category')

In [69]:
# Train/test split
np.random.seed(0)
msk = np.random.rand(len(df)) < 0.8
tr = df[msk]
te = df[~msk]

# Make convenience dfs
x_train = tr.drop('system_type',axis=1)
y_train = tr.system_type
x_test = te.drop('system_type',axis=1)
y_test = te.system_type

In [72]:
param = {'num_leaves': 31, 'objective': 'multi'}
model = lgb.LGBMClassifier(seed=0)
model.fit(x_train,y_train,eval_set=[(x_test,y_test),(x_train,y_train)],verbose=0,eval_metric='logloss')

/home/at/.pyenv/versions/3.9.5/envs/audette-env/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/at/.pyenv/versions/3.9.5/envs/audette-env/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/at/.pyenv/versions/3.9.5/envs/audette-env/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


LGBMClassifier(seed=0)

In [83]:
qf = pd.read_csv("questions.csv", index_col="id")
qf.assigned_schedule = qf.assigned_schedule.astype('category')
Yf = qf['system_type']
Xf = qf.drop('system_type', axis=1)
preds = model.predict(Xf)

In [84]:
display(Yf)
display(preds)

id
1    ELECTRIC HEATING
2           AC SYSTEM
3         VENTILATION
Name: system_type, dtype: object

array(['ELECTRIC HEATING', 'AC SYSTEM', 'VENTILATION'], dtype=object)